In [9]:
import pickle
from connections.mysql_connector import MySQL_Connector
from models.topic_modeling import Topic_Modeling
from connections.neo4j_connector import Neo4j_Connector
import os
from datetime import datetime
from gensim import corpora, models, similarities
from models.graph_generator import Graph_Generator
from models.tuple_extractor import Tuple_Extractor
from acessos import read, get_conn, persistir_uma_linha, persistir_multiplas_linhas, replace_df, update_banco
import pandas as pd
import re
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.parsing.preprocessing import strip_punctuation
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import numpy as np
from tqdm.auto import tqdm  # for notebooks
import math

In [10]:
nlp = spacy.load("pt_core_news_lg")

connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)

#conn = connector.return_conn("influencer_br")
tuple_extractor = Tuple_Extractor()

In [11]:
def transformar_componentes(df_tuplas):
    features = [atr for atr in list(df_tuplas.columns) if atr not in {"embedding_id","sentenca_id","tupla_id", "label"}]
    componentes = df_tuplas.loc[:, features].values
    return componentes


def get_df_agrupamento(tag_metodo):
    query = '''SELECT a.*, video_id 
        FROM agrupamento a 
        INNER JOIN sentencas using(sentenca_id) 
        WHERE 
            tag_metodo = '{}' AND 
            num_tuplas_candidatas <> -1'''.format(tag_metodo)

    df_agrupamento = read(conn, query)
    return df_agrupamento

def get_df_tuplas(table_name, sentenca_id):
    query = f'''select * from {table_name} where sentenca_id = {sentenca_id} and label="tupla_completa" '''
    df_tuplas = read(conn, query)
    return df_tuplas

def gerar_modelo_kmeans(componentes, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(componentes)
    return kmeans.labels_
    
def criar_grupos_tuplas(num_tuplas_candidatas, tag_metodo, sentenca_id, video_id):
    #retorna um dict entre o index do grupo e id da tabela
    
    for index in range(num_tuplas_candidatas):
        dict_data = {
            "sentenca_id": [sentenca_id],
            "num_grupo": [index],
            "tag_metodo": [tag_metodo],
            "video_id": [video_id]
        }
        print(index)
        replace_df(pd.DataFrame(dict_data), "grupo_resumo",conn)
    
    query = f'''SELECT * 
                FROM 
                    grupo_resumo 
                WHERE 
                    video_id = '{video_id}' AND 
                    tag_metodo = '{tag_metodo}' AND 
                    sentenca_id = '{sentenca_id}'   
            '''
    df_grupos = read(conn, query)
    
    dict_de_para ={}
    for row in df_grupos.iterrows(): 
        grupo_id = row[1]['grupo_id']
        num_grupo = row[1]['num_grupo']
        dict_de_para[num_grupo] = grupo_id 
        
    return df_grupos, dict_de_para
    

Informe o método a ser processado:
tsne_distortion
Informe a tabela origem (tsne/embedding):
tsne
Processando sentença 3 do vídeo --K2l7qd7W0


NameError: name 'dict_de_para' is not defined

## Montando Grupos de Tuplas 

In [12]:
def criar_grupo(tag_metodo, sentenca_id, video_id, num_grupo):
    dict_data = {
        "tag_metodo": [tag_metodo],
        "sentenca_id": [sentenca_id],
        "video_id": [video_id],
        "num_grupo": [num_grupo]
    }
    df_grupo = pd.DataFrame(dict_data)
    return df_grupo

def get_df_tuplas(table_name, sentenca_id):
    query = f'''select * from {table_name} where sentenca_id = {sentenca_id} and label="tupla_completa" '''
    df_tuplas = read(conn, query)
    return df_tuplas

def transformar_componentes(df_tuplas):
    features = [atr for atr in list(df_tuplas.columns) if atr not in {"embedding_id","sentenca_id","tupla_id", "label"}]
    componentes = df_tuplas.loc[:, features].values
    return componentes

def buscar_grupos(tag_metodo, sentenca_id):
    query = f'''SELECT * FROM grupo_resumo WHERE tag_metodo = "{tag_metodo}" and sentenca_id = {sentenca_id};'''
    return read(conn, query)

In [13]:
connector = MySQL_Connector("conn_orfeu")
conn = connector.return_conn(db_name)

#embedding_distortion
print("Informe o método a ser processado:")
tag_metodo = input()

print("Informe a tabela origem (tsne/embedding):")
table_name = input()


df_agrupamento = get_df_agrupamento(tag_metodo)

#Iterando Setenças
for row in df_agrupamento.iterrows():
    sentenca_id = row[1]['sentenca_id']
    num_tuplas_candidatas = row[1]['num_tuplas_candidatas']
    tag_metodo = row[1]['tag_metodo']
    video_id = row[1]['video_id']
    
    print("Processando a sent: {}".format(sentenca_id))
    
    #Criando os grupos
    print("   -> Criando Grupos:")
    for num_grupo in range(0, num_tuplas_candidatas):
        df_grupo = criar_grupo(tag_metodo, sentenca_id, video_id, num_grupo)
        status = replace_df(df_grupo,"grupo_resumo", conn)
    
    df_grupos_criados = buscar_grupos(tag_metodo, sentenca_id)    

    #Buscando tuplas das sentenças
    df_tuplas = get_df_tuplas(table_name, sentenca_id) 

    
    componentes = transformar_componentes(df_tuplas)
    labels = gerar_modelo_kmeans(componentes, num_tuplas_candidatas) 
    
    df_tuplas['label'] = labels
    df_tuplas['label'] = df_tuplas['label'].astype(int)
    
   
    df_tupla_grupo =  pd.merge(df_tuplas,
                    df_grupos_criados,
                    how='inner',
                    left_on=['label', 'sentenca_id'],
                    right_on=['num_grupo', 'sentenca_id']
                   )
    df_tupla_grupo = df_tupla_grupo[['tupla_id','grupo_id']]
    print("   -> Adicionando Tupla Grupo:")
    lista_erros = []
    
    df_tupla_grupo['grupo_id'] = df_tupla_grupo['grupo_id'].astype(str)
    df_tupla_grupo['tupla_id'] = df_tupla_grupo['tupla_id'].astype(str)
    
    print("*************")
    print(df_tupla_grupo.dtypes)
    print(df_tupla_grupo.head())
    print("*****%%******")
    
    status = replace_df(df_tupla_grupo,"tupla_grupo", conn)
    if status == -1:
        lista_erros.append(sentenca_id)
        break
        
        

Informe o método a ser processado:
tsne_distortion_grupo_100
Informe a tabela origem (tsne/embedding):
tsne
Processando a sent: 4
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0       39    41969
1       40    41969
2       42    41969
3       44    41969
4       45    41969
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 6
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0       62    41973
1       63    41970
2       65    41970
3       64    41971
4       66    41971
*****%%*****

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0      972    42027
1      979    42027
2      973    42025
3      975    42025
4      977    42025
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 64
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0      998    42028
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 69
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     1043    42029
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 70
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso 

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     1638    42086
1     1639    42086
2     1640    42086
3     1655    42086
4     1676    42086
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 113
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     1985    42094
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 117
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     1997    42095
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 119
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Suces

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     2338    42146
1     2343    42146
2     2339    42148
3     2340    42148
4     2344    42148
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 154
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     2349    42152
1     2350    42149
2     2353    42149
3     2351    42151
4     2352    42151
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 155
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     2360    42153
*****%%*

Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     2648    42193
1     2649    42193
2     2656    42193
3     2657    42193
4     2659    42193
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 211
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0     2665    42198
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 214
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0 

...

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    19197    43187
1    19198    43187
2    19199    43187
3    19201    43187
4    19203    43187
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1332
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    19207    43190
1    19210    43190
2    19211    43190
3    19208    43192
4    19212    43192
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1335
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  t

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    19510    43232
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1401
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    19513    43241
1    19514    43241


   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    20414    43281
1    20415    43281
2    20416    43281
3    20417    43284
4    20418    43282
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1454
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    20425    43285
1    20426    43285
2    20427    43285
3    20428    43285
4    20429    43286
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1456
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Pers

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    20800    43333
1    20801    43333
2    20802    43335
3    20805    43335
4    20803    43331
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1517
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    20812    43336
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1520
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    20821    43337
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1524
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grup

   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    21124    43381
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1573
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
   -> Adicionando Tupla Grupo:
*************
tupla_id    object
grupo_id    object
dtype: object
  tupla_id grupo_id
0    21129    43382
*****%%******
Iniciando Persistencia
Sucesso na inserção
Processando a sent: 1574
   -> Criando Grupos:
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
Iniciando Persistencia
Sucesso na inserção
In

In [19]:
df_tupla_grupo['grupo_id'] = df_tupla_grupo['grupo_id'].astype(str)
df_tupla_grupo['tupla_id'] = df_tupla_grupo['tupla_id'].astype(str)
print(df_tupla_grupo.dtypes)
df_tupla_grupo

tupla_id    object
grupo_id    object
dtype: object


,tupla_id,grupo_id
0,86,39610


In [20]:
# grupo_id = ["1","1","3"]
# tag_metodo = ['teste','teste','teste']
# sentenca_id = ["1","1","3"]
# num_grupo = ["1","1","3"]

# grupo_id = [1,1,3]
# tag_metodo = ["teste","teste","teste"]
# sentenca_id = [1,1,3]
# num_grupo = [1,1,3]

grupo_id = [1]
tag_metodo = ["teste"]
sentenca_id = [1]
num_grupo = [1]


aux = pd.DataFrame({"tag_metodo": tag_metodo, "sentenca_id":sentenca_id,"num_grupo":num_grupo})

status = replace_df(df_tupla_grupo,"grupo_resumo_teste", conn)

Iniciando Persistencia
Infelizmente algo deu errado na inserção (persistir_uma_linha)
(1054, "Unknown column 'tupla_id' in 'field list'")


In [70]:
aux

,grupo_id,tag_metodo,sentenca_id,num_grupo
0,1,teste,1,1
1,1,teste,1,1
2,3,teste,3,3
